In [125]:
import numpy as np
import pandas as pd

In [126]:
articles = pd.read_csv('articles.csv')
reddit = pd.read_csv('reddit_posts.csv')

In [127]:
reddit.head()

,title,text,link
0,Meme Mondays,NaN,https://www.reddit.com//r/datascience/comments...
1,Friendly reminder not to work too hard. You'll...,The year just started and there are already ov...,https://www.reddit.com//r/datascience/comments...
2,Data scientist quits her job at Spotify,In summary and basically talks about how she w...,https://www.reddit.com//r/datascience/comments...
3,XKCD Comic does machine learning,NaN,https://www.reddit.com//r/datascience/comments...
4,I investigated the Underground Economy of Glas...,Online company reviews are high stakes.\n\nTop...,https://www.reddit.com//r/datascience/comments...


In [128]:
reddit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 994 entries, 0 to 993
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   994 non-null    object
 1   text    893 non-null    object
 2   link    994 non-null    object
dtypes: object(3)
memory usage: 23.4+ KB


In [129]:
reddit = reddit.dropna().reset_index()

In [130]:
import re

import nltk

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
custom_stop_words = ['ds', 'data', 'data science', 'science', 'u', 'like', 'would', 'one', 'also', 'even', 'much', 'hey']

def text_processing(text):
    
    text = str(text).replace('\n', ' ')
    text = str(text).replace('e.g.', ' ')
    
    text = re.sub('http\S+', '', text)
    text = re.sub('-', '', text)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    text = text.lower()
    
    text = ' '.join([word for word in text.split() if word not in (stop_words)])
    text = ''.join([word for word in text if not word.isdigit()])

    for word in custom_stop_words:
        text = re.sub(r'\b{}\b'.format(word), '', text)

    return text

<>:15: DeprecationWarning: invalid escape sequence '\S'
<>:15: DeprecationWarning: invalid escape sequence '\S'
C:\Users\thoma\AppData\Local\Temp\ipykernel_2468\3878923509.py:15: DeprecationWarning: invalid escape sequence '\S'
  text = re.sub('http\S+', '', text)


In [131]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatization(text):
    text = text_processing(text)
    
    text = nltk.word_tokenize(text)
            
    text = [lemmatizer.lemmatize(word) for word in text]
    text = ' '.join(word for word in text)
    
    return text

reddit['text'] = reddit['text'].apply(lambda x: lemmatization(x))
articles['text'] = articles['text'].apply(lambda x: lemmatization(x))

In [132]:
reddit.text[3]

'world director make k vp anything make k world compensation increase become smaller higher promotion present completely achievable following path reality take lot luck politics become anything higher manager happens rarely'

# Keyword Extraction

In [133]:
import yake

kw_extractor = yake.KeywordExtractor()

for word, num in kw_extractor.extract_keywords(reddit.text[4]):
    print(type(reddit.text[4]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [134]:
import gensim 
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim

In [141]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        new = [word for word, num in kw_extractor.extract_keywords(' '.join(new))]
        final.append(new)
    return (final)
data_words = gen_words(reddit.text)


In [142]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)



In [143]:
lda_model  = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word, num_topics = 10, random_state =100, update_every = 1,
                                            passes = 10, alpha = 'auto')

In [144]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds = 'mmds', R=10)
vis

C:\Users\thoma\anaconda3\Lib\site-packages\sklearn\manifold\_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.154548  0.093827       1        1  11.331341
6      0.045216  0.160541       2        1  10.750642
0     -0.131880 -0.109268       3        1  10.723365
1     -0.095949  0.152246       4        1  10.338370
8      0.059827 -0.171605       5        1  10.017478
4     -0.063168  0.060939       6        1   9.935687
2     -0.026494 -0.119919       7        1   9.476027
3     -0.167331  0.001348       8        1   9.406703
7      0.067369 -0.021726       9        1   9.224176
9      0.157861 -0.046384      10        1   8.796211, topic_info=                            Term      Freq     Total Category  logprob  \
969                      working  3.000000  3.000000  Default  10.0000   
4631                     curious  3.000000  3.000000  Default   9.0000   
2372                     thought  4.000000  4.000000  Default   8.0000   
1043                   scientist  7.000000  7.000000  Default   7.0000   
810                          job  7.000000  7.000000  Default   6.0000   
...                          ...       ...       ...      ...      ...   
2317                        work  0.522125  2.474769  Topic10  -7.9705   
7508                     company  0.521673  2.522593  Topic10  -7.9714   
1023  machine learning algorithm  0.520745  2.032275  Topic10  -7.9731   
3614   machine learning engineer  0.519902  2.458543  Topic10  -7.9748   
3611           learning engineer  0.519880  1.478627  Topic10  -7.9748   

      loglift  
969   10.0000  
4631   9.0000  
2372   8.0000  
1043   7.0000  
810    6.0000  
...       ...  
2317   0.8749  
7508   0.8548  
1023   1.0692  
3614   0.8772  
3611   1.3856  

[163 rows x 6 columns], token_table=       Topic      Freq                         Term
term                                               
12642      9  1.012238   advice greatly appreciated
14821      4  0.984407                analyst basic
7446       4  0.984406  anecdotal confirmation bias
2434       6  0.676275                    applicant
2434      10  0.676275                    applicant
...      ...       ...                          ...
1863       8  0.332563                         year
1405       3  0.653801         year work experience
1405       5  0.653801         year work experience
14742      3  0.975659             year worked year
10411      7  1.005459                youtube video

[224 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 1, 2, 9, 5, 3, 4, 8, 10])